# PEC Monte Carlo simulation

Start by importing MagmaPEC and MagmaPandas and any other packages you want to use. Here we also import Pandas for importing pressure data. For details on the use of MagmaPandas, please see it's [documentation](https://magmapandas.readthedocs.io/en/latest/).

In [1]:
import MagmaPEC as mpc
import MagmaPandas as mp

import pandas as pd

Confirm the model and PEC configurations. If you want to change models or PEC settings, follow the [configuration example](https://magmapec.readthedocs.io/en/latest/notebooks/config.html).

In [2]:
print(mpc.model_configuration)
print(mpc.PEC_configuration)


################## MagmaPandas ###################
##################################################
General settings__________________________________
fO2 buffer.....................................QFM
ΔfO2.............................................1
Melt Fe3+/Fe2+.............................sun2024
Kd Fe-Mg ol-melt........................toplis2005
Melt thermometer....................putirka2008_15
Volatile solubility model.......iaconomarziano2012
Volatile species.............................mixed
##################################################


############ Post-entrapment crystallisation ############
################### correction model ####################
Settings_________________________________________________
Fe2+ behaviour...................................buffered
Stepsize equilibration (moles)...................0.002   
Stepsize crystallisation (moles).................0.05    
Decrease factor..................................5       
FeO convergence (wt. %).......

In the next few steps we import all relevant data and set up the melt initial FeO prediction model. These steps are identical to the [FeOi](https://magmapec.readthedocs.io/en/latest/notebooks/FeOi.html#) and [PEC correction](https://magmapec.readthedocs.io/en/latest/notebooks/pec_corr.html#) examples

Import melt inclusion and olivine data:

In [3]:
melt_file = "./data/melt.csv"
olivine_file = "./data/olivine.csv"

melt = mp.read_melt(melt_file, index_col=["name"])
olivine = mp.read_olivine(olivine_file, index_col=["name"])

Import inclusion internal pressures or calculate them if you have measured melt CO2 (and H2O). See the [PEC model example](https://magmapec.readthedocs.io/en/latest/notebooks/pec_corr.html) for details on how to do the calculation. Here we import them from a file.

In [4]:
pressure_file ="./data/pressure.csv"
pressure = pd.read_csv(pressure_file, index_col = ["name"]).squeeze()

Set up the melt initial FeO prediction model:

In [5]:
wholerock_file = "./data/wholerock.csv"
wholerock = mp.read_melt(wholerock_file, index_col=["name"])

x = wholerock.drop(columns=["FeO"])
FeOi_predict = mpc.FeOi_prediction(x=x, FeO=wholerock["FeO"])

do_not_use = ["MnO", "P2O5", "Cr2O3", "total"]

model_fits = FeOi_predict.calculate_model_fits(exclude=do_not_use)
FeOi_predict.select_predictors(idx=3)

Next, we need to set up the object that handles the random sampling of errors in the Monte Carlo simulation. This is done with the `PEC_MC_parameters` class and it includes the following parameters for error propagation:

- **melt_errors**
        
    propagate errors on melt composition by providing one standard deviation errors per element as a pandas Series (fixed errors for all inclusions) or DataFrame (errors per inclusion).

- **olivine_errors**

    propagate errors on olivine composition by providing one standard deviation errors per element as a pandas Series (fixed errors for all inclusions) or DataFrame (errors per inclusion).

- **FeOi_errors**

    propagate errors on estimate melt initial FeO contents. Fixed errors can be provided either for the whole dataset, or per inclusion. Alternatively, an [FeOi_prediction object](https://magmapec.readthedocs.io/en/latest/notebooks/FeOi.html) can be provided to propagate errors on predictions models.

- **Fe3Fe2**

    Propagate errors on modelled melt Fe<sup>2+</sup>/Fe<sup>3+</sup> ratios. Errors are automatically calculated by MagmaPandas based on the selected model. Pass *True* to this parameter to activate it.

- **Kd**

    Propagate errors on modelled olivine-melt Fe-Mg partition coefficients. Errors are automatically calculated by MagmaPandas based on the selected model. Pass *True* to this parameter to activate it.

By default errors are not propagated - you explicitely need to tell MagmaPEC to do so when initialising the `PEC_MC_parameters` object

In this example we will use all error propagation options, which means we need to provide melt and olivine composition errors. We import these from .csv files containing error data for individual inclusions and olivines. This is just an example with randomly generated errors, normally you should use analytical errors.

In [6]:
melt_errors_file = "./data/melt_errors.csv"
olivine_errors_file = "./data/olivine_errors.csv"

melt_errors = pd.read_csv(melt_errors_file, index_col=[0])
olivine_errors = pd.read_csv(olivine_errors_file, index_col=[0])

Make very sure that the elements in the error data have identical sorting to the melt and olivine dataframes, otherwise errors will be applied to the wrong elements. 
We can force this by sorting the columns of the error dataframes (or series) with the *elements* attributes of the melt and olivine MagmaFrames:

In [7]:
melt_errors = melt_errors[melt.elements]
olivine_errors = olivine_errors[olivine.elements]

Here's what they look like:

In [8]:
melt_errors.head()

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl
PI032-04-01,1.02,0.64,0.14,0.47,0.69,0.10,0.08,0.05,0.18,0.05,0.05,0.17,0.21,0.22,0.10,0.12
PI032-04-02,1.06,0.84,0.29,0.46,0.54,0.05,0.03,0.01,0.06,0.18,0.04,0.24,0.12,0.13,0.12,0.04
PI041-02-02,1.04,0.90,0.09,0.55,0.49,0.22,0.21,0.01,0.16,0.04,0.00,0.15,0.23,0.04,0.17,0.06
PI041-03-01,0.98,0.64,0.10,0.40,0.53,0.12,0.12,0.08,0.01,0.03,0.00,0.05,0.36,0.06,0.14,0.04
PI041-03-03,1.02,0.54,0.31,0.42,0.68,0.18,0.03,0.02,0.11,0.06,0.10,0.18,0.08,0.13,0.06,0.15


In [9]:
olivine_errors.head()

,SiO2,FeO,MgO,NiO,MnO,Al2O3,CaO
PI032-04-01,1.06,0.60,1.22,0.01,0.09,0.08,0.02
PI032-04-02,0.85,0.57,1.28,0.14,0.07,0.24,0.06
PI041-02-02,1.11,0.44,1.30,0.08,0.10,0.01,0.19
PI041-03-01,0.99,0.50,1.31,0.14,0.16,0.15,0.01
PI041-03-03,0.95,0.64,1.21,0.13,0.11,0.10,0.15


Together with the `FeOi_prediction` object, we pass these as arguments to the `PEC_MC_parameters` object. We also set *Fe3Fe2* and *Kd* to *True* in order to propagate their model errors.

In [10]:
mc_parameters = mpc.PEC_MC_parameters(melt_errors=melt_errors, olivine_errors=olivine_errors, FeOi_errors=FeOi_predict, Fe3Fe2=True, Kd=True, temperature=True)

Now we can create the Monte Carlo model with the `PEC_MC` object

In [11]:
pec_mc_model = mpc.PEC_MC(inclusions=melt, olivines=olivine, P_bar=pressure, FeO_target=FeOi_predict, MC_parameters=mc_parameters)

In [12]:
pec_mc_model.run(n=100)

Monte Carlo iterations... |█████████████████████████| 100% [100/100] in 5:34.9 


Results are stored internally in the following attributes:

- `pec`: pandas DataFrame

    Average PEC extents (%) of the MC model and their one standard deviation errors. Positive values indicate post-entrapment crystallisation and negative melting.

- `inclusions_corr`: MagmaPandas Melt frame

    Averages of corrected melt inclusion compositions (wt. %)

- `inclusions_stddev`: pandas DataFrame

    One standard deviation errors on inclusions_corr (wt. %)

- `pec_MC`: pandas DataFrame

    PEC extents for individual iterations. Positive values indicate post-entrapment crystallisation and negative melting.

- `inclusions_MC`: dictionary of MagmaPandas Melt frames

    corrected melt inclusion compositions for individual iterations.

In [13]:
pec = pec_mc_model.pec
inclusions_corrected = pec_mc_model.inclusions_corr
inclusions_errors = pec_mc_model.inclusions_stddev

pec_mc = pec_mc_model.pec_MC
inclusions_MC = pec_mc_model.inclusions_MC

In [14]:
pec

,pec,stddev
name,,
PI032-04-01,10.142907,4.548620
PI032-04-02,11.357208,4.516174
PI041-02-02,0.815962,2.905620
PI041-03-01,13.601006,4.891701
PI041-03-03,12.990314,5.096770
PI041-05-04,-3.929812,2.600279
PI041-05-06,2.448095,2.897591
PI041-07-01,12.192332,4.100629
PI041-07-02,11.642825,4.584894


In [15]:
inclusions_corrected

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl
name,,,,,,,,,,,,,,,,
PI032-04-01,48.929506,13.950077,7.792052,9.689972,10.266758,3.591971,0.674746,0.143971,2.433775,0.267235,0.000000,0.577367,1.385620,0.107006,0.131049,0.058895
PI032-04-02,48.387886,14.527454,7.716546,9.402522,10.373694,3.442483,0.908129,0.142659,2.561196,0.331248,0.000000,0.619907,1.274149,0.091349,0.171556,0.049222
PI041-02-02,49.110997,16.852401,5.010501,9.070138,10.286066,3.768048,1.071387,0.155178,2.809909,0.557633,0.000000,0.449459,0.653586,0.045766,0.120576,0.038356
PI041-03-01,45.908601,15.496055,7.465648,10.864778,10.709174,3.299626,1.126627,0.117502,3.109843,0.541264,0.000000,0.788303,0.333445,0.086844,0.101473,0.050817
PI041-03-03,45.205636,15.769103,7.280743,11.136857,10.804693,3.380074,1.155561,0.091417,3.245381,0.518059,0.000000,0.845972,0.303047,0.099356,0.075014,0.089088
PI041-05-04,47.767635,18.666835,3.740278,9.468179,9.349547,4.629541,1.605357,0.163083,2.500323,0.829164,0.000000,0.537715,0.460330,0.103547,0.119798,0.058667
PI041-05-06,46.300992,16.988870,4.802190,8.919516,11.476623,4.004677,1.458893,0.199312,3.632750,0.622349,0.000000,0.646548,0.634686,0.110768,0.135974,0.065852
PI041-07-01,45.806451,15.253737,7.230335,9.672404,11.574336,3.142135,1.269885,0.144395,3.525818,0.554700,0.000000,0.473358,1.003248,0.098693,0.162401,0.088104
PI041-07-02,45.799498,15.572251,7.040141,10.028869,11.435917,3.202950,1.385644,0.137894,3.516662,0.613457,0.000000,0.369843,0.619599,0.073364,0.143599,0.060312


In [16]:
inclusions_errors

,SiO2_stddev,Al2O3_stddev,MgO_stddev,CaO_stddev,FeO_stddev,Na2O_stddev,K2O_stddev,MnO_stddev,TiO2_stddev,P2O5_stddev,Cr2O3_stddev,CO2_stddev,H2O_stddev,F_stddev,S_stddev,Cl_stddev
name,,,,,,,,,,,,,,,,
PI032-04-01,0.775498,0.746274,1.347776,0.548462,0.387428,0.170826,0.080350,0.045789,0.172639,0.047309,0.000000,0.166161,0.151126,0.120183,0.092257,0.066645
PI032-04-02,0.662486,0.974221,1.340132,0.540367,0.280666,0.139232,0.043591,0.013957,0.101987,0.159243,0.000000,0.204853,0.116350,0.083550,0.097586,0.031455
PI041-02-02,0.801771,0.852027,0.977585,0.620597,0.295437,0.253802,0.223687,0.016399,0.142956,0.040269,0.000000,0.124794,0.214493,0.034622,0.127440,0.044981
PI041-03-01,0.635252,0.793534,1.509443,0.489334,0.171090,0.154564,0.115539,0.070082,0.122906,0.031724,0.000000,0.062085,0.243744,0.050101,0.103684,0.034254
PI041-03-03,0.599366,0.791975,1.536312,0.594178,0.202218,0.232998,0.054251,0.024905,0.132212,0.055846,0.000000,0.157376,0.082809,0.087225,0.049402,0.096005
PI041-05-04,0.659694,0.746251,0.800943,0.406394,0.250304,0.182255,0.141593,0.142476,0.073743,0.032151,0.000000,0.176663,0.169449,0.099098,0.011033,0.010877
PI041-05-06,0.794695,1.008854,0.902820,0.386829,0.342837,0.368856,0.237313,0.160696,0.169971,0.049422,0.000000,0.210679,0.295429,0.091452,0.059496,0.025937
PI041-07-01,0.642131,0.829666,1.231199,0.447094,0.220794,0.138080,0.054678,0.067932,0.144302,0.026866,0.000000,0.050375,0.169976,0.091470,0.028705,0.103810
PI041-07-02,0.669416,0.851325,1.378383,0.441312,0.194065,0.197043,0.167482,0.052329,0.137774,0.023256,0.000000,0.178564,0.210634,0.002670,0.048546,0.059283


In [17]:
pec_mc.head()

name,PI032-04-01,PI032-04-02,PI041-02-02,PI041-03-01,PI041-03-03,PI041-05-04,PI041-05-06,PI041-07-01,PI041-07-02,PI052-01-02
iteration,,,,,,,,,,
0,9.491785,12.37854,0.558154,17.356738,9.883386,-5.203918,7.117175,12.336694,11.452002,-5.500684
1,6.483826,6.706494,0.007953,11.806006,8.736792,-4.8,-0.384712,10.688623,11.50802,-7.511267
2,10.872607,10.0,1.6,14.381372,11.568558,-4.991931,2.610034,11.288184,12.0047,-7.929797
3,16.288306,14.915552,3.818115,20.182031,17.168396,-1.710498,6.012329,17.928955,17.4979,-3.990601
4,5.41156,7.172974,-2.197412,5.799561,5.962061,-6.129272,-2.251938,9.605444,5.729797,-12.054614


The dataframes in `inclusions_MC` also have *isothermal_equilibration*, *Kd_equilibration*, and *FeO_converge* columns. These columns show if equilibrations during stage [1] and [2] and FeO convergence in stage [2] were successful. Extreme cases of random error sampling may yield melt-olivine pairs that cannot be equilibrated without requiring crystallising exceeding the mass of the inclusion inclusion or exchange of more Mg or Fe than the inclusion contains. If that is the case, no corrected compositions are calculated and the isothermal- of Kd-equilibration column is set to *False*.

In [18]:
inclusions_MC["PI032-04-01"].head()

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl,isothermal_equilibration,Kd_equilibration,FeO_converge
iteration,,,,,,,,,,,,,,,,,,,
0,48.966977,14.379993,7.589772,10.360526,9.725173,3.502419,0.623934,0.152254,2.308284,0.30396,0.0,0.416633,1.304399,0.073733,0.224818,0.067124,True,True,True
1,49.224937,14.129253,6.81813,10.346858,10.074029,3.893835,0.729172,0.161464,2.460332,0.27356,0.0,0.606776,1.11106,0.028237,0.142357,0.0,True,True,True
2,49.050058,13.41574,8.317149,9.659579,10.324613,3.552607,0.700479,0.189927,2.393232,0.217943,0.0,0.543845,1.401281,0.0,0.233547,0.0,True,True,True
3,48.396895,12.802653,9.897628,9.594693,9.885725,3.492379,0.707936,0.126148,2.218483,0.304289,0.0,0.826447,1.362872,0.240757,0.143096,0.0,True,True,True
4,49.642055,13.908098,6.479232,10.016678,10.470928,3.689569,0.615221,0.10899,2.502613,0.277505,0.0,0.563363,1.251678,0.118311,0.277896,0.077865,True,True,True
